In [1]:
import os
import shutil
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow import keras
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

class_names = ['0', '1', '2', '3', 
               '4', '5', '6', '7', 
               '8', '9', 'A', 'B', 
               'C', 'D', 'E', 'F', 
               'G', 'H', 'K', 'L', 
               'M', 'N', 'O', 'P', 
               'R', 'S', 'T', 'U', 
               'V', 'X', 'Y', 'Z']

datagen = ImageDataGenerator(
    rescale=1.0/255, 
    validation_split=0.2,
)

train_generator = datagen.flow_from_directory(
    "../Training_set/charTrainset", 
    target_size=(28,12), 
    batch_size=20,
    subset='training',
    class_mode='sparse'
)

validation_generator = datagen.flow_from_directory(
    "../Training_set/charTrainset", 
    target_size=(28,12), 
    batch_size=20,
    subset='validation',
    class_mode='sparse'
)

Found 1729 images belonging to 32 classes.
Found 417 images belonging to 32 classes.


In [2]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', 
                           padding='same',
                           input_shape=(28, 12, 3)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu',
                           padding='same'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', 
                           padding='same'),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu',
                           padding='same'),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu',
                           padding='same'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='softmax')
])

2022-01-06 23:33:11.146703: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-06 23:33:11.147077: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.


In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 12, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 12, 64)        36928     
_________________________________________________________________
dropout (Dropout)            (None, 28, 12, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 6, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 6, 128)        73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 6, 128)        147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 6, 128)        1

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00003),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
class stopCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}) :
        if (logs.get('accuracy') > 0.98) :
            print('\nReached 98% accuracy so stopping training')
            self.model.stop_training = True

In [6]:
callback = stopCallback()
model.fit(train_generator, epochs=200, callbacks=[callback])

Train for 87 steps
Epoch 1/200
87/87 [==============================] - 29s 330ms/step - loss: 3.4191 - accuracy: 0.0717
Epoch 2/200
87/87 [==============================] - 23s 265ms/step - loss: 3.2341 - accuracy: 0.0960
Epoch 3/200
87/87 [==============================] - 22s 258ms/step - loss: 3.0051 - accuracy: 0.1695
Epoch 4/200
87/87 [==============================] - 23s 270ms/step - loss: 2.5867 - accuracy: 0.3042
Epoch 5/200
87/87 [==============================] - 23s 264ms/step - loss: 2.2463 - accuracy: 0.3840
Epoch 6/200
87/87 [==============================] - 22s 257ms/step - loss: 1.9177 - accuracy: 0.4656
Epoch 7/200
87/87 [==============================] - 23s 261ms/step - loss: 1.7394 - accuracy: 0.4974
Epoch 8/200
87/87 [==============================] - 23s 267ms/step - loss: 1.5597 - accuracy: 0.5523
Epoch 9/200
87/87 [==============================] - 25s 283ms/step - loss: 1.4125 - accuracy: 0.5830
Epoch 10/200
87/87 [==============================] - 23s 266ms

In [ ]:
model.evaluate(validation_generator, verbose=1)

In [ ]:
path1 = "../Testing_set/greenparkingchar"
path2 = "../Testing_set/greenparking2char"

all_path = [path1, path2]

final = "../charLabel/"

i = 1

for path in all_path:
    for file in os.listdir(path) :
        image_mp= mpimg.imread(path + '/' + file) / 255
        image_mp = np.expand_dims(image_mp, axis=0)
        shutil.copyfile(path + '/' + file, final + class_names[np.argmax(model.predict(image_mp))] + '/' + file)
        i += 1
        if i == 10:
            break
    if i == 10:
        break